In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import os
import seaborn as sns
from tqdm.auto import tqdm
import math

/scratch/work/lyub2/.conda_envs/lb-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 新尝试

In [2]:
# Projected coordinate system
MAP_CRS = 'EPSG:3857'
CAL_CRS = 'EPSG:3067'

# EPSG 4326 uses a coordinate system the same as a GLOBE (curved surface). 
# EPSG 3857 uses a coordinate system the same as a MAP (flat surface).
LONLAT_CRS = 'EPSG:4326'

In [3]:
gdf_areas = gpd.read_file(r'/scratch/work/lyub2/HZR/ykr_250_15_kuntaa_grid_v2.geojson').to_crs(CAL_CRS)

In [26]:
gdf_areas.columns

Index(['id', 'geometry'], dtype='object')

### 计算功能定义

In [4]:
def plot_areas(areas):
    plt.figure(figsize=(5, 5), dpi=150)
    ax = plt.subplot(1, 1, 1)
    gdf_areas[gdf_areas['id'].isin(areas)].plot(ax=ax, ec='k')
    #plt.show()

In [5]:
def euclidean_distance(x1, y1, x2, y2):
    if pd.isnull(x2) or pd.isnull(y2):
        return None
    return ((x2 - x1)**2 + (y2 - y1)**2)**0.5

def grids_to_fill_gap(distance, gap_limit=360, grid_size=250):
    if distance <= gap_limit:
        return 0
    effective_gap = distance - gap_limit
    grids_needed = math.ceil(effective_gap / grid_size)
    return grids_needed

def calculate_diff_distances(gdf_tc):
    if 'geometry' not in gdf_tc.columns or not isinstance(gdf_tc['geometry'], gpd.GeoSeries):
        raise ValueError("The geometry column is missing or is not a GeoSeries in gdf_tc.")
    
    gdf_tc['centroid'] = gdf_tc['geometry'].centroid
    gdf_tc['x'] = gdf_tc['centroid'].x
    gdf_tc['y'] = gdf_tc['centroid'].y

    gdf_tc['x_next'] = gdf_tc['x'].shift(-1)
    gdf_tc['y_next'] = gdf_tc['y'].shift(-1)
    gdf_tc['area_id_next'] = gdf_tc['area_id'].shift(-1)

    gdf_tc['diff_dist'] = gdf_tc.apply(
        lambda row: euclidean_distance(row['x'], row['y'], row['x_next'], row['y_next']),
        axis=1
    )
    return gdf_tc

def calculate_areas_next_to_each_other(gdf_tc, gap_limit=360):
    areas_next = gdf_tc[gdf_tc['diff_dist'] <= gap_limit].dropna(subset=['diff_dist'])
    adjacent_pairs = areas_next.apply(
        lambda row: {
            'index': row.name,
            'area_id_current': row['area_id'],
            'area_id_next': row['area_id_next'],
            'distance': row['diff_dist']
        },
        axis=1
    ).tolist()
    return adjacent_pairs

def calculate_areas_with_gaps(gdf_tc, gap_limit=360, grid_size=250):
    gdf_tc_gap = gdf_tc[gdf_tc['diff_dist'] > gap_limit].dropna(subset=['diff_dist'])
    if gdf_tc_gap.empty:
        return []
    else:
        gdf_tc_gap['grids_needed'] = gdf_tc_gap['diff_dist'].apply(
            lambda d: grids_to_fill_gap(d, gap_limit, grid_size)
        )
        gaps = gdf_tc_gap.apply(
            lambda row: {
                'index': row.name,
                'area_id_current': row['area_id'],
                'area_id_next': row['area_id_next'],
                'distance': row['diff_dist'],
                'grids_needed': row['grids_needed']
            },
            axis=1
        ).tolist()
        return gaps

def measure_distance_gap(df_tc):
    GAP_LIMIT = 360
    GRID_SIZE = 250

    gdf_tc = gdf_areas.merge(df_tc, left_on='id', right_on='area_id')
    gdf_tc = gpd.GeoDataFrame(gdf_tc, geometry='geometry') ## 新加的
    gdf_tc = gdf_tc.sort_values('area_order').reset_index(drop=True)

    gdf_tc = calculate_diff_distances(gdf_tc)

    #print("gdf_tc['diff_dist'] ",gdf_tc['diff_dist'])
    dist_total = gdf_tc['diff_dist'].sum() 

    dist_adj = gdf_tc[gdf_tc['diff_dist'] <= GAP_LIMIT]['diff_dist'].sum()
    gap_ratio = ((dist_total - dist_adj) / dist_total) if dist_total else 0
    dist_count = gdf_tc['diff_dist'].count()

    adjacent_pairs = calculate_areas_next_to_each_other(gdf_tc, GAP_LIMIT)
    gaps = calculate_areas_with_gaps(gdf_tc, GAP_LIMIT, GRID_SIZE)

    dist_gap_min = gdf_tc[gdf_tc['diff_dist'] > GAP_LIMIT]['diff_dist'].min()
    dist_gap_max = gdf_tc[gdf_tc['diff_dist'] > GAP_LIMIT]['diff_dist'].max() 
    dist_gap_mean = gdf_tc[gdf_tc['diff_dist'] > GAP_LIMIT]['diff_dist'].mean() 
    dist_gap_count = gdf_tc[gdf_tc['diff_dist'] > GAP_LIMIT]['diff_dist'].count() 

    areas = set([x['area_id_current'] for x in adjacent_pairs] + [x['area_id_next'] for x in adjacent_pairs])
    area_next_to_each_other = len(areas)

    areas_with_gaps = np.sum([x['grids_needed'] for x in gaps])
    
    # print(f"Total distance: {dist_total}") #完整的包含gap和没有gap的距离计算结果
    # print(f"Adjacent distance: {dist_adj}")#相邻的area的总长度
    # #print(f"Total intervals: {dist_count}")
    # print(f"Adjacent/Total distance ratio: {round(dist_adj / dist_total * 100, 2) if dist_total else 0}") #这个leg里 非gap区域的距离占比
    # print(f"Gap min distance: {dist_gap_min}")
    # print(f"Gap max distance: {dist_gap_max}")
    # print(f"Gap mean distance: {dist_gap_mean}")
    # print(f"Gap count: {dist_gap_count}") #这个leg里一共有几段gap
    # print(f"Number of adjacent pairs: {len(adjacent_pairs)}")
    # #print(f"Number of gaps: {len(gaps)}")
    # print(f"Areas next to each other: {area_next_to_each_other}") #这个leg里所有有相邻的areas的数量
    # print(f"Areas with gaps: {areas_with_gaps}") #要填补这个leg里所有gap需要的格子数量
    
    # print("Gaps:")
    # for gap in gaps:
    #     print(gap)
    # print("\n")
    
    # areas = gdf_tc['area_id'].astype(int).values
    # plot_areas(areas)

    #return adjacent_pairs, gaps 
    return adjacent_pairs, gaps, dist_total, dist_adj, gap_ratio, dist_gap_min, dist_gap_max, dist_gap_mean, dist_gap_count, area_next_to_each_other

In [87]:
group = sorted_bus.head(1000).groupby("temp_device")
for temp_device, df in group:
    if len(df.leg_order.unique())>1:
        print(temp_device,len(df.leg_order.unique()) )

000OM3CXMpmd 2
0027AfcePA3w 3
0028zaGIy9ki 3
003za0UCaFe4 2
004rm1ePFm5i 2
005kboCECfb0 2
005yJIA58wyD 2


In [90]:
test_df = group.get_group("000OM3CXMpmd").reset_index(drop=True)
test_df

,temp_device,tripchain_id,leg_order,area_id,area_order,mode
0,000OM3CXMpmd,0.0,3.0,64989.0,12.0,BUS
1,000OM3CXMpmd,0.0,3.0,64991.0,13.0,BUS
2,000OM3CXMpmd,0.0,3.0,64299.0,14.0,BUS
3,000OM3CXMpmd,0.0,3.0,63709.0,15.0,BUS
4,000OM3CXMpmd,0.0,3.0,63092.0,16.0,BUS
...,...,...,...,...,...,...
57,000OM3CXMpmd,1.0,1.0,62477.0,29.0,BUS
58,000OM3CXMpmd,1.0,1.0,62476.0,30.0,BUS
59,000OM3CXMpmd,1.0,1.0,63092.0,31.0,BUS
60,000OM3CXMpmd,1.0,1.0,63709.0,32.0,BUS


In [96]:
test_df['td_tc_lo'] = test_df["temp_device"]+"_"+ test_df["tripchain_id"].astype(str)+"_"+test_df["leg_order"].astype(str)
test_df

,temp_device,tripchain_id,leg_order,area_id,area_order,mode,td_tc_lo
0,000OM3CXMpmd,0.0,3.0,64989.0,12.0,BUS,000OM3CXMpmd_0.0_3.0
1,000OM3CXMpmd,0.0,3.0,64991.0,13.0,BUS,000OM3CXMpmd_0.0_3.0
2,000OM3CXMpmd,0.0,3.0,64299.0,14.0,BUS,000OM3CXMpmd_0.0_3.0
3,000OM3CXMpmd,0.0,3.0,63709.0,15.0,BUS,000OM3CXMpmd_0.0_3.0
4,000OM3CXMpmd,0.0,3.0,63092.0,16.0,BUS,000OM3CXMpmd_0.0_3.0
...,...,...,...,...,...,...,...
57,000OM3CXMpmd,1.0,1.0,62477.0,29.0,BUS,000OM3CXMpmd_1.0_1.0
58,000OM3CXMpmd,1.0,1.0,62476.0,30.0,BUS,000OM3CXMpmd_1.0_1.0
59,000OM3CXMpmd,1.0,1.0,63092.0,31.0,BUS,000OM3CXMpmd_1.0_1.0
60,000OM3CXMpmd,1.0,1.0,63709.0,32.0,BUS,000OM3CXMpmd_1.0_1.0


In [85]:
test_df =sorted_bus[sorted_bus.temp_device=="000OM3CXMpmd"].reset_index(inplace=True)
test_df

In [86]:
test_df

In [ ]:
def identify_mode(z):
    if not pd.isna(z['line_type']):
        return z['line_type']
    else:
        return z['activity']

## 获取输入数据

### 试这个

In [ ]:
df_tc_01 = pd.read_csv('/scratch/work/lyub2/Problem_output_September/P4/df_p41.csv', usecols= ['temp_device', 'tripchain_id', 'leg_order', 'activity', 'line_type', 'area_order', 'area_id'])
df_tc = df_tc_01.dropna(subset=['leg_order', 'activity', 'line_type'], how='all').reset_index(drop=True)

/tmp/ipykernel_3286701/3541719597.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tc_01 = pd.read_csv('/scratch/work/lyub2/Problem_output_September/P4/df_p41.csv', usecols= ['temp_device', 'tripchain_id', 'leg_order', 'activity', 'line_type', 'area_order', 'area_id'])


In [ ]:
df = pd.read_csv('/scratch/work/lyub2/Problem_output_September/P4/df_p41.csv', usecols= ['temp_device', 'tripchain_id', 'leg_order', 'activity', 'line_type', 'area_order', 'area_id'])
df_tc_02 = df_tc_02.dropna(subset=['leg_order', 'activity', 'line_type'], how='all').reset_index(drop=True)

In [ ]:
def identify_mode(z):
    if not pd.isna(z['line_type']):
        return z['line_type']
    else:
        return z['activity']
        
df_tc['mode'] = df_tc.apply(identify_mode, axis=1)

In [ ]:
df_walking = df_tc_01[df_tc_01['activity'] == 'WALKING']

In [ ]:
df_walking

,temp_device,tripchain_id,leg_order,area_id,area_order,activity,line_type
111,jelLoknH162b,0.0,1.0,56789.0,2.0,WALKING,NaN
112,jelLoknH162b,0.0,1.0,56788.0,3.0,WALKING,NaN
123,yIKqMeFibPXa,0.0,0.0,58100.0,0.0,WALKING,NaN
124,yIKqMeFibPXa,0.0,0.0,63707.0,1.0,WALKING,NaN
126,GFeSeZhojxwm,0.0,0.0,65824.0,1.0,WALKING,NaN
...,...,...,...,...,...,...,...
32772884,V8h4G6Gc7Ski,0.0,0.0,34017.0,40.0,WALKING,NaN
32773014,uKNIxzkZWsPN,0.0,0.0,52425.0,0.0,WALKING,NaN
32773015,uKNIxzkZWsPN,0.0,0.0,50488.0,1.0,WALKING,NaN
32773309,ecKVb5tmuCTJ,0.0,1.0,48517.0,28.0,WALKING,NaN


In [ ]:
#df_tc_test = df_tc.head(200000)

In [ ]:
folder_path = '/scratch/work/lyub2/Problem_output_September'
file_names = [f'df_p{i}.csv' for i in range(41, 47)]

filtered_dfs = []

for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    print(file_path)

print(file_names)

/scratch/work/lyub2/Problem_output_September/df_p41.csv
/scratch/work/lyub2/Problem_output_September/df_p42.csv
/scratch/work/lyub2/Problem_output_September/df_p43.csv
/scratch/work/lyub2/Problem_output_September/df_p44.csv
/scratch/work/lyub2/Problem_output_September/df_p45.csv
/scratch/work/lyub2/Problem_output_September/df_p46.csv
['df_p41.csv', 'df_p42.csv', 'df_p43.csv', 'df_p44.csv', 'df_p45.csv', 'df_p46.csv']


### 非公共交通的

In [ ]:

folder_path = '/scratch/work/lyub2/Problem_output_September/P4'
file_names = [f'df_p{i}.csv' for i in range(41, 47)]

filtered_dfs = []

for file_name in tqdm(file_names):
    file_path = os.path.join(folder_path, file_name)
    
    df = pd.read_csv(file_path, usecols= ['temp_device', 'tripchain_id', 'leg_order', 'activity', 'line_type', 'area_order', 'area_id'])
    df = df.dropna(subset=['leg_order', 'activity'], how='any').reset_index(drop=True)
        
    df_filtered = df[df['activity'].notnull() & df['line_type'].isnull()]
    filtered_dfs.append(df_filtered)

df_nonPT = pd.concat(filtered_dfs, ignore_index=True)
df_nonPT = df_nonPT.drop(columns=['line_type'])
df_nonPT.rename(columns={'activity': 'mode'}, inplace=True)

print('start saving file~')
df_nonPT.to_csv('/scratch/work/lyub2/Problem_output_September/P4/df_nonPT.csv', index=False)
print('DONE!')

  0%|          | 0/6 [00:00<?, ?it/s]

/tmp/ipykernel_517226/2379088468.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols= ['temp_device', 'tripchain_id', 'leg_order', 'activity', 'line_type', 'area_order', 'area_id'])
/tmp/ipykernel_517226/2379088468.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols= ['temp_device', 'tripchain_id', 'leg_order', 'activity', 'line_type', 'area_order', 'area_id'])


start saving file~
DONE!


In [ ]:
df_nonPT

,temp_device,tripchain_id,leg_order,area_id,area_order,mode
0,jelLoknH162b,0.0,0.0,56792.0,0.0,IN_VEHICLE
1,jelLoknH162b,0.0,0.0,56791.0,1.0,IN_VEHICLE
2,jelLoknH162b,0.0,1.0,56789.0,2.0,WALKING
3,jelLoknH162b,0.0,1.0,56788.0,3.0,WALKING
4,jelLoknH162b,1.0,0.0,64989.0,0.0,IN_VEHICLE
...,...,...,...,...,...,...
78209284,PsVhPwUuuBI4,0.0,0.0,27558.0,16.0,WALKING
78209285,PsVhPwUuuBI4,0.0,1.0,27558.0,16.0,WALKING
78209286,w15wDe1fb2sI,0.0,3.0,70001.0,3.0,WALKING
78209287,FfPUD6vxsNrN,0.0,0.0,54049.0,2.0,WALKING


### 公共交通的

In [ ]:
folder_path = '/scratch/work/lyub2/Problem_output_September/P4'
file_names = [f'df_p{i}.csv' for i in range(41, 47)]

filtered_dfs = []

for file_name in tqdm(file_names):
    file_path = os.path.join(folder_path, file_name)
    
    df = pd.read_csv(file_path, usecols= ['temp_device', 'tripchain_id', 'leg_order', 'line_type', 'area_order', 'area_id'])
    df = df.dropna(subset=['leg_order', 'line_type'], how='any').reset_index(drop=True)
    filtered_dfs.append(df)

df_PT = pd.concat(filtered_dfs, ignore_index=True)
df_PT.rename(columns={'line_type': 'mode'}, inplace=True)

print('start saving file~')
df_PT.to_csv('/scratch/work/lyub2/Problem_output_September/P4/df_PT.csv', index=False)
print('DONE!')

  0%|          | 0/6 [00:00<?, ?it/s]

/tmp/ipykernel_517226/1244946812.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols= ['temp_device', 'tripchain_id', 'leg_order', 'line_type', 'area_order', 'area_id'])
/tmp/ipykernel_517226/1244946812.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols= ['temp_device', 'tripchain_id', 'leg_order', 'line_type', 'area_order', 'area_id'])


start saving file~
DONE!


In [ ]:
df_PT

,temp_device,tripchain_id,leg_order,area_id,area_order,mode
0,mLBH9BYePp6u,0.0,3.0,34709.0,76.0,BUS
1,mLBH9BYePp6u,0.0,3.0,34709.0,76.0,BUS
2,mLBH9BYePp6u,0.0,3.0,30375.0,87.0,BUS
3,mLBH9BYePp6u,0.0,3.0,30375.0,87.0,BUS
4,mLBH9BYePp6u,0.0,3.0,29017.0,92.0,BUS
...,...,...,...,...,...,...
56098619,ciJcewM0mr8V,1.0,0.0,63092.0,4.0,BUS
56098620,r5qosTdRWpF3,2.0,1.0,55661.0,36.0,BUS
56098621,G6IvMSV94Zi6,0.0,1.0,53509.0,11.0,BUS
56098622,UjVdCou5qOMZ,1.0,1.0,67621.0,18.0,BUS


In [ ]:
df_p4.to_csv('/scratch/work/lyub2/Problem_output_September/P4/df_p4.csv', index=False)

## 终于可以跑功能了！

In [6]:
df_PT = pd.read_csv('/scratch/work/lyub2/Problem_output_September/P4/df_PT.csv')

In [6]:
df_nonPT = pd.read_csv('/scratch/work/lyub2/Problem_output_September/P4/df_nonPT.csv')

In [9]:
print(df_PT['mode'].unique())
print(df_nonPT['mode'].unique())

['BUS' 'TRAIN' 'SUBWAY' 'TRAM' 'UBUS' 'FERRY']
['IN_VEHICLE' 'WALKING' 'RUNNING' 'UNKNOWN' 'ON_BICYCLE' 'BEACON']


In [8]:
df_bus

,temp_device,tripchain_id,leg_order,area_id,area_order,mode
0,mLBH9BYePp6u,0.0,3.0,34709.0,76.0,BUS
1,mLBH9BYePp6u,0.0,3.0,34709.0,76.0,BUS
2,mLBH9BYePp6u,0.0,3.0,30375.0,87.0,BUS
3,mLBH9BYePp6u,0.0,3.0,30375.0,87.0,BUS
4,mLBH9BYePp6u,0.0,3.0,29017.0,92.0,BUS
...,...,...,...,...,...,...
56098619,ciJcewM0mr8V,1.0,0.0,63092.0,4.0,BUS
56098620,r5qosTdRWpF3,2.0,1.0,55661.0,36.0,BUS
56098621,G6IvMSV94Zi6,0.0,1.0,53509.0,11.0,BUS
56098622,UjVdCou5qOMZ,1.0,1.0,67621.0,18.0,BUS


In [15]:
#大宝贝！终于出来了！

def get_change_indices(df):
    # Identify where the 'id' changes
    df['td_tc_lo'] = df["temp_device"]+"_"+ df["tripchain_id"].astype(str)+"_"+df["leg_order"].astype(str)
    change_indices = df.index[df['td_tc_lo'] != df['td_tc_lo'].shift()].tolist()
    # Exclude the first index since we only want where changes occur after the start
    
    return change_indices

def calculate_legs_onfire(df):

    result =  []

    print('Start sorting')
    sorted_df = df.sort_values(['temp_device', 'tripchain_id', 'leg_order', 'area_order']).reset_index(drop=True)
    print('Sorting done')
    
    change_indices = get_change_indices(sorted_df)
    change_indices.append(len(sorted_df))
    
    

    for i in tqdm(range(len(change_indices)-1)):
        # Extract row values
        
        df_temp = sorted_df.iloc[change_indices[i]:change_indices[i+1]].reset_index(drop=True)
        temp_device,tripchain_id,leg_order = df_temp.td_tc_lo[0].split("_")
 
        mode = df_temp["mode"][0]
        
        # obtain data from measure_distance_gap function.
        adjacent_pairs, gaps, dist_total, dist_adj, gap_ratio, dist_gap_min, dist_gap_max, dist_gap_mean, dist_gap_count, area_next_to_each_other  = measure_distance_gap(df_temp)

        result.append({
            'Mode' : mode,
            'Temp' : temp_device,
            'Tripchain ID' : tripchain_id,
            'Leg Order' : leg_order,
            'Total distance': dist_total,
            'Adjacent distance': dist_adj,
            'Gap ratio': gap_ratio,
            'Gap min distance': dist_gap_min,
            'Gap max distance': dist_gap_max,
            'Gap mean distance': dist_gap_mean,
            'Gap count': dist_gap_count, 
            'Areas next to each other':area_next_to_each_other
            })


    print('Convert to df and saving as csv')
    result = pd.DataFrame(result)
    result.to_csv('/scratch/work/lyub2/Problem_output_September/P4/P4_ferry.csv',index = False)
    return pd.DataFrame(result)


In [15]:
P4_car = calculate_legs_onfire(df_car)

Start sorting


: 

In [10]:
P4_train = calculate_legs_onfire(df_train)

Start sorting
Sorting done


  0%|          | 0/254588 [00:00<?, ?it/s]

Convert to df and saving as csv


In [14]:
P4_ubus = calculate_legs_onfire(df_ubus)

Start sorting
Sorting done


  0%|          | 0/2908 [00:00<?, ?it/s]

Convert to df and saving as csv


In [16]:
P4_ferry = calculate_legs_onfire(df_ferry)

Start sorting
Sorting done


  0%|          | 0/1110 [00:00<?, ?it/s]

Convert to df and saving as csv


In [137]:
tram_test = calculate_legs_onfire(df_tram.head(300))

Start sorting
Sorting done


  0%|          | 0/29 [00:00<?, ?it/s]

Convert to DF


In [12]:
P4_tram = calculate_legs_onfire(df_tram)

Start sorting
Sorting done


  0%|          | 0/142390 [00:00<?, ?it/s]

Convert to df and saving as csv


In [11]:
P4_run = calculate_legs_onfire(df_run)

Start sorting
Sorting done


  0%|          | 0/107903 [00:00<?, ?it/s]

Convert to df and saving as csv


In [10]:
P4_bike= calculate_legs_onfire(df_bike)

Start sorting
Sorting done


  0%|          | 0/423830 [00:00<?, ?it/s]

Convert to df and saving as csv


In [9]:
P4_unknown = calculate_legs_onfire(df_unknown)

Start sorting
Sorting done


  0%|          | 0/463121 [00:00<?, ?it/s]

Convert to df and saving as csv


In [11]:
P4_beacon = calculate_legs_onfire(df_beacon)

Start sorting
Sorting done


  0%|          | 0/1 [00:00<?, ?it/s]

Convert to df and saving as csv


In [12]:
df_beacon

,temp_device,tripchain_id,leg_order,area_id,area_order,mode
6470419,fGrP1kYqgYwx,0.0,1.0,64985.0,3.0,BEACON
6470422,fGrP1kYqgYwx,0.0,1.0,64989.0,1.0,BEACON
6470423,fGrP1kYqgYwx,0.0,1.0,64987.0,2.0,BEACON
6810010,fGrP1kYqgYwx,0.0,1.0,65817.0,4.0,BEACON


## 分析 + 表格总结